<a href="https://colab.research.google.com/github/HSubbu/dtc-data-engineering/blob/main/etl_python_Apr22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime


In [28]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-04-25 14:09:26--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip.1’

datasource.zip.1    100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-04-25 14:09:26 (436 MB/s) - ‘datasource.zip.1’ saved [4249/4249]



In [29]:
! unzip datasource.zip -d dealership_data

Archive:  datasource.zip
replace dealership_data/used_car_prices1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


In [30]:
tmpfile    = "dealership_temp.tmp"               # store all extracted data

logfile    = "dealership_logfile.txt"            # all event logs will be stored

targetfile = "dealership_transformed_data.csv"   # transformed data is stored

In [31]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

In [32]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

In [33]:
def extract_from_xml(file_to_process):

    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])

    tree = ET.parse(file_to_process) 

    root = tree.getroot() 

    for person in root: 

        car_model = person.find("car_model").text 

        year_of_manufacture = int(person.find("year_of_manufacture").text)

        price = float(person.find("price").text) 

        fuel = person.find("fuel").text 

        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True) 

        return dataframe

In [34]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f: f.write(timestamp + ',' + message + 'n')

In [35]:
# import required module
import glob

# assign directory
directory = '/content/dealership_data'
csv_df = pd.DataFrame()
# iterate over files in
# that directory
for filename in glob.iglob(f'{directory}/*'):
  if 'csv' in filename:
    df = extract_from_csv(filename)
    csv_df=csv_df.append(df)
print(csv_df.shape)



(30, 4)


In [36]:
json_df = pd.DataFrame()
# iterate over files in
# that directory
for filename in glob.iglob(f'{directory}/*'):
  if 'json' in filename:
    df = extract_from_json(filename)
    json_df=json_df.append(df)
print(json_df.shape)

(30, 4)


In [37]:
xml_df = pd.DataFrame()
# iterate over files in
# that directory
for filename in glob.iglob(f'{directory}/*'):
  if 'xml' in filename:
    df = extract_from_xml(filename)
    xml_df=xml_df.append(df)
print(xml_df.shape)

(3, 4)


In [38]:
merged_df = csv_df.append(json_df).append(xml_df) # merge all dataframes
log("Extraction phase completed")

In [39]:
merged_df.head(3)

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol


In [40]:
A#Step 3(Transform):round off price
merged_df['price'] = merged_df['price'].apply(lambda x : round(x,2))
log("Transform phase completed")

In [41]:
#We’ve now gone through the steps of extracting, transforming, and loading data from various sources into a single target file.
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

In [42]:
load("cars.csv",merged_df)
log("Loading completed")

In [45]:
with open("/content/dealership_logfile.txt") as f:
  print(f.read())

14:10:02-Apr-25-2022,Extraction phase completedn14:10:03-Apr-25-2022,Transform phase completedn14:10:03-Apr-25-2022,Loading completedn
